In [27]:
# Load some test data
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.datasets import make_classification
bank = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bank.csv')
bank.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [28]:
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

# Drop missing values from embarked
bank = bank.dropna().copy()

# Encode our features and target as needed
features = ["age", "job", "marital", "education", "default", "housing", "loan", "contact", "month", "day_of_week", "campaign", "pdays", "previous", "poutcome", "emp.var.rate", "cons.price.idx","cons.conf.idx", "euribor3m", "nr.employed"]
X = pd.get_dummies(bank[features], drop_first=True)
y = bank['y']

# Create a SMOTE object and fit it to the data
smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X, y)

# Check the class distribution before and after SMOTE
print("Original class distribution:", pd.Series(y).value_counts())
print("Resampled class distribution:", pd.Series(y_resampled).value_counts())

# Split our data into training and test data, with 30% reserved for testing
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.30)

# Build the decision tree
clf = RandomForestClassifier()

# Train it
clf.fit(X_train, y_train)

# Test it
clf.score(X_test, y_test)

# Note that this gives us an accuracy score, which may not be the best metric.
# See the SciKit-Learn docs for more ways to assess a model's performance, as
# well as methods for cross validation.

Original class distribution: y
no     32861
yes     4208
Name: count, dtype: int64
Resampled class distribution: y
no     32861
yes    32861
Name: count, dtype: int64


0.935233554800426

In [29]:
test = pd.read_csv("https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bank_holdout_test_mini.csv")

# Do same transformations as on the training set
test = pd.get_dummies(test[features], drop_first=True) # Apply get_dummies to test data
test = test.reindex(columns=X_train.columns, fill_value=0) # Reorder and add missing columns
predictions = clf.predict(test)

# Convert the predictions to a dataframe and label the column 'predictions'
my_predictions = pd.DataFrame(predictions, columns = ['predictions'])

#map yes and no to 1 and 0
my_predictions['predictions'] = my_predictions['predictions'].map({'yes': 1, 'no': 0})
# Replace PUTTEAMNUMBERHERE with your team
my_predictions.to_csv("team6-module2-predictions.csv",index=False)